In [ ]:
import numpy as np
einsum = np.einsum

In [ ]:
X = np.random.random((1000,2))
e = np.array([1.,1.])

In [ ]:
# (X*X)@ e
%timeit Y=(X*X)@ e

In [ ]:
# einsum("ni,ni->n", X, X)
%timeit Y=einsum("ni,ni->n", X, X)

In [ ]:
X=np.array([[1.,1.],[2.,2.],[3.,3.]])

In [ ]:
a=np.array([0.2,0.3])

In [ ]:
(X @ a)[:,None] * X

In [ ]:
np.array([[1,2],[2,3],[3,4]]).T * np.array([10,20,30])

In [ ]:
import ctypes
A = np.ascontiguousarray(np.arange(100))
A1 = A[2:-2]

In [ ]:
%load_ext cython

In [ ]:
%%cython

import numpy as np

cdef double[::1] buf = np.arange(30, dtype="d")
cdef double[::1] buf1 = buf[10:20]
sub = np.asarray(buf1, copy=False)
print(sub)
sub[0] = 100
print(np.asarray(buf, copy=False))
for i in range(10):
    print(buf1[i], end=' ')
print()
for i in range(30):
    print(buf[i], end=' ')


In [1]:
from mlgrad.list_values import list_double

In [2]:
a = list_double()
a.extend([1,2,3,4,5,6,7,8])
b = a.asarray()
print(b)
b /= b.sum()
print(a.aslist())
for i in range(len(a)):
    print(a[i], end=' ')

[1. 2. 3. 4. 5. 6. 7. 8.]
[0.027777777777777776, 0.05555555555555555, 0.08333333333333333, 0.1111111111111111, 0.1388888888888889, 0.16666666666666666, 0.19444444444444445, 0.2222222222222222]
0.027777777777777776 0.05555555555555555 0.08333333333333333 0.1111111111111111 0.1388888888888889 0.16666666666666666 0.19444444444444445 0.2222222222222222 

In [9]:
import numpy as np
from mlgrad.inventory import empty_int_array2

In [4]:
ai = empty_int_array2(10)

In [6]:
ai.fill(0)

In [7]:
ai

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [10]:
a = np.zeros(10, "i")
a

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)